In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from collections import defaultdict

pd.options.display.max_colwidth = None

In [2]:
df = pd.read_csv('data/ML-CUP23-TR.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         1000 non-null   int64  
 1    INPUT_0   1000 non-null   float64
 2    INPUT_1   1000 non-null   float64
 3    INPUT_2   1000 non-null   float64
 4    INPUT_3   1000 non-null   float64
 5    INPUT_4   1000 non-null   float64
 6    INPUT_5   1000 non-null   float64
 7    INPUT_6   1000 non-null   float64
 8    INPUT_7   1000 non-null   float64
 9    INPUT_8   1000 non-null   float64
 10   INPUT_9   1000 non-null   float64
 11   TARGET_x  1000 non-null   float64
 12   TARGET_y  1000 non-null   float64
 13   TARGET_z  1000 non-null   float64
dtypes: float64(13), int64(1)
memory usage: 109.5 KB


In [4]:
df.describe()

,ID,INPUT_0,INPUT_1,INPUT_2,INPUT_3,INPUT_4,INPUT_5,INPUT_6,INPUT_7,INPUT_8,INPUT_9,TARGET_x,TARGET_y,TARGET_z
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,500.500000,-0.201135,0.188475,-0.317963,0.609508,0.669649,0.339157,0.107086,0.432494,-0.302264,-0.046340,-11.286752,-15.720560,17.092153
std,288.819436,0.839704,0.877978,0.853533,0.711304,0.569266,0.704837,0.511980,0.696107,0.642846,0.758187,26.261142,29.102229,10.263310
min,1.000000,-0.998595,-0.997432,-0.999699,-0.997935,-0.984356,-0.967577,-0.903815,-0.996894,-0.984569,-0.999776,-81.418755,-82.387726,0.028556
25%,250.750000,-0.977303,-0.937027,-0.997872,0.712357,0.681167,-0.323789,-0.333222,-0.067024,-0.883347,-0.920071,-27.232117,-42.160871,8.051677
50%,500.500000,-0.680112,0.776463,-0.977509,0.995674,0.948389,0.651689,0.170296,0.832223,-0.547592,0.042152,1.043153,-1.598209,16.981470
75%,750.250000,0.858231,0.984984,0.760654,0.999555,0.988074,0.990822,0.544865,0.980276,0.279711,0.712157,8.021954,8.032845,25.872736
max,1000.000000,0.999299,0.998755,0.998443,0.999946,0.998807,0.999678,0.946637,0.999542,0.990203,0.988618,24.213701,23.510721,34.992752


In [5]:
df.head()

,ID,INPUT_0,INPUT_1,INPUT_2,INPUT_3,INPUT_4,INPUT_5,INPUT_6,INPUT_7,INPUT_8,INPUT_9,TARGET_x,TARGET_y,TARGET_z
0,1,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,2,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,3,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,4,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,5,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251


In [6]:
df.tail()

,ID,INPUT_0,INPUT_1,INPUT_2,INPUT_3,INPUT_4,INPUT_5,INPUT_6,INPUT_7,INPUT_8,INPUT_9,TARGET_x,TARGET_y,TARGET_z
995,996,-0.803560,-0.878859,-0.978391,0.957539,0.996655,0.996924,-0.226305,0.097814,-0.922666,0.934153,15.389553,-41.068806,27.513502
996,997,0.718841,0.995748,-0.942678,0.999929,0.747763,-0.808726,0.176132,0.999100,-0.753970,-0.969009,-36.228770,13.067430,11.672133
997,998,-0.977912,-0.971108,0.956233,-0.979524,0.222033,0.986609,0.658273,-0.987310,0.937697,0.143420,7.265506,-53.497242,2.815666
998,999,0.284803,-0.988684,-0.427197,0.883317,0.993302,0.999500,-0.019456,-0.648110,-0.955231,0.901298,5.545274,-63.348396,27.989340
999,1000,-0.987580,0.993488,-0.998944,0.999703,0.936830,-0.564176,0.262568,0.892081,-0.198204,0.494586,6.160610,8.321016,13.021444
